In [ ]:
import pyroSAR
from pyroSAR.snap.auxil import parse_recipe, parse_node
from pyroSAR.snap.auxil import gpt, groupbyWorkers

from os.path import join, exists, basename
from os import makedirs
from shutil import rmtree
from glob import glob

import warnings
from time import perf_counter
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
def create_master_gpt(out_gpt, in_slc, out_slc, subswath, bursts):

    workflow = parse_recipe('blank')

    read = parse_node('Read')
    read.parameters['file'] = in_slc
    workflow.insert_node(read)
    
    topsar_split = parse_node('TOPSAR-Split')
    topsar_split.parameters['subswath'] = subswath
    topsar_split.parameters['firstBurstIndex'] = bursts[0]
    topsar_split.parameters['lastBurstIndex'] = bursts[1]
    workflow.insert_node(topsar_split, before=read.id)
    
    apply_orbit = parse_node('Apply-Orbit-File')
    apply_orbit.parameters['orbitType'] = 'Sentinel Precise (Auto Download)'
    apply_orbit.parameters['polyDegree'] = 3
    apply_orbit.parameters['continueOnFail'] = 'false'
    workflow.insert_node(apply_orbit, before=topsar_split.id)
    
    write = parse_node('Write')
    write.parameters['file'] = out_slc
    write.parameters['formatName'] = 'GeoTIFF-BigTIFF'
    workflow.insert_node(write, before=apply_orbit.id)

    workflow.write(out_gpt)

In [ ]:
def create_coherence_gpt(out_gpt, in_slc_0, in_slc_one, in_slc_two, dem_path, out_filename, subswath, bursts, **kwargs):
    
    workflow = parse_recipe('blank')
    
    # 0th branch - Read common master
    read_0 = parse_node('Read')
    read_0.parameters['file'] = in_slc_0
    workflow.insert_node(read_0)
    
    # 1st branch - preprocess first slc
    read_one = parse_node('Read')
    read_one.parameters['file'] = in_slc_one
    workflow.insert_node(read_one)
    
    topsar_split_one = parse_node('TOPSAR-Split')
    topsar_split_one.parameters['subswath'] = subswath
    topsar_split_one.parameters['firstBurstIndex'] = bursts[0][0]
    topsar_split_one.parameters['lastBurstIndex'] = bursts[0][1]
    workflow.insert_node(topsar_split_one, before=read_one.id)
    
    apply_orbit_one = parse_node('Apply-Orbit-File')
    apply_orbit_one.parameters['orbitType'] = 'Sentinel Precise (Auto Download)'
    apply_orbit_one.parameters['polyDegree'] = 3
    apply_orbit_one.parameters['continueOnFail'] = 'false'
    workflow.insert_node(apply_orbit_one, before=topsar_split_one.id)
    
    # 2nd branch - preprocess second slc
    read_two = parse_node('Read')
    read_two.parameters['file'] = in_slc_two
    workflow.insert_node(read_two)
    
    topsar_split_two = parse_node('TOPSAR-Split')
    topsar_split_two.parameters['subswath'] = subswath
    topsar_split_two.parameters['firstBurstIndex'] = bursts[0][0]
    topsar_split_two.parameters['lastBurstIndex'] = bursts[0][1]
    workflow.insert_node(topsar_split_two, before=read_two.id)
    
    apply_orbit_two = parse_node('Apply-Orbit-File')
    apply_orbit_two.parameters['orbitType'] = 'Sentinel Precise (Auto Download)'
    apply_orbit_two.parameters['polyDegree'] = 3
    apply_orbit_two.parameters['continueOnFail'] = 'false'
    workflow.insert_node(apply_orbit_two, before=topsar_split_two.id)
    
    # combined branch
    # coregister preprocessed slcs from all three branches
    back_geocoding = parse_node('Back-Geocoding')
    back_geocoding.parameters['demName'] = 'External DEM'
    back_geocoding.parameters['demResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    back_geocoding.parameters['externalDEMFile'] = dem_path
    back_geocoding.parameters['externalDEMNoDataValue'] = 0.0
    back_geocoding.parameters['resamplingType'] = 'BILINEAR_INTERPOLATION'
    back_geocoding.parameters['maskOutAreaWithoutElevation'] = 'false'
    back_geocoding.parameters['outputRangeAzimuthOffset'] = 'false'
    back_geocoding.parameters['outputDerampDemodPhase'] = 'false'
    back_geocoding.parameters['disableReramp'] = 'false'
    workflow.insert_node(back_geocoding, before=[read_0.id, apply_orbit_one.id, apply_orbit_two.id])

    # select only bands from the two subsequent slave images
    band_select = parse_node('BandSelect')
    band_select.parameters['bandNamePattern'] = '.*slv.*'
    workflow.insert_node(band_select, before=back_geocoding.id)
    
    # estimate coherence
    coherence = parse_node('Coherence')
    coherence.parameters['cohWinAz'] = 2
    coherence.parameters['cohWinRg'] = 10
    #coherence.parameters['substractFlatEarthPhase'] = 'false'
    coherence.parameters['srpPolynomialDegree'] = 5
    coherence.parameters['srpNumberPoints'] = 501
    coherence.parameters['orbitDegree'] = 3
    coherence.parameters['squarePixel'] = 'true'
    coherence.parameters['subtractTopographicPhase'] = 'false'
    coherence.parameters['demName'] = 'External DEM'
    #coherence.parameters['demResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    coherence.parameters['externalDEMFile'] = dem_path
    coherence.parameters['externalDEMNoDataValue'] = 0.0
    coherence.parameters['externalDEMApplyEGM'] = 'true'
    coherence.parameters['tileExtensionPercent'] = 100
    coherence.parameters['singleMaster'] = 'true'
    workflow.insert_node(coherence, before=band_select.id)

    deburst = parse_node('TOPSAR-Deburst')
    workflow.insert_node(deburst, before=coherence.id)

    tc = parse_node('Terrain-Correction')
    tc.parameters['demName'] = 'External DEM'
    tc.parameters['demResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    tc.parameters['externalDEMFile'] = dem_path
    tc.parameters['externalDEMNoDataValue'] = 0.0
    tc.parameters['externalDEMApplyEGM'] = 'true'

    tc.parameters['imgResamplingMethod'] = 'BILINEAR_INTERPOLATION'
    tc.parameters['pixelSpacingInMeter'] = 0.0
    tc.parameters['pixelSpacingInDegree'] = 0.0
    tc.parameters['mapProjection'] = '''GEOGCS["WGS 84", DATUM["World Geodetic System 1984",
        SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], 
        AUTHORITY["EPSG","6326"]], 
        PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], 
        UNIT["degree", 0.017453292519943295], 
        AXIS["Geodetic longitude", EAST], 
        AXIS["Geodetic latitude", NORTH], 
        AUTHORITY["EPSG","4326"]]'''
    # maybe? so everything is aligned
    tc.parameters['alignToStandardGrid'] = 'true'
    tc.parameters['standardGridOriginX'] = 0.0
    tc.parameters['standardGridOriginY'] = 0.0
    tc.parameters['nodataValueAtSea'] = 'false'
    tc.parameters['saveDEM'] = 'false'
    tc.parameters['saveLatLon'] = 'false'
    tc.parameters['saveIncidenceAngleFromEllipsoid'] = 'false'
    tc.parameters['saveLocalIncidenceAngle'] = 'false'
    tc.parameters['saveProjectedLocalIncidenceAngle'] = 'false'
    tc.parameters['saveSelectedSourceBand'] = 'true'
    tc.parameters['saveLayoverShadowMask'] = 'false'
    #tc.parameters['outputComplex'] = 'false'
    tc.parameters['applyRadiometricNormalization'] = 'false'
    tc.parameters['saveSigmaNought'] = 'false'
    tc.parameters['saveGammaNought'] = 'false'
    tc.parameters['saveBetaNought'] = 'false'
    tc.parameters['incidenceAngleForSigma0'] = 'Use projected local incidence angle from DEM'
    tc.parameters['incidenceAngleForGamma0'] = 'Use projected local incidence angle from DEM'
    tc.parameters['auxFile'] = 'Latest Auxiliary File'
    workflow.insert_node(tc, before=deburst.id)

    write = parse_node('Write')
    write.parameters['file'] = out_filename
    write.parameters['formatName'] = 'GeoTIFF-BigTIFF'
    workflow.insert_node(write, before=tc.id)

    workflow.write(out_gpt)

### Run processing in a loop

In [ ]:
# Define directiories used
ROOT_DIR = '/media/sf_JD/DP'

dtm_path = join(ROOT_DIR, 'DTM/DMR_4G_4326.tif')
temp_dir = join(ROOT_DIR, 'temp')
xmls_dir = join(temp_dir, 'snap_xmls')
makedirs(xmls_dir) if not exists(xmls_dir) else print(f'{xmls_dir} already exists.')

In [ ]:
## old, did not wortk for orbits 22 and 95 - the burst indices are not constant accross the whole year
topsar_split_params = {
    22: {
        'subswath': 'IW2',
        'burst_first': 1,
        'burst_last': 2},
    73: {
        'subswath': 'IW1',
        'burst_first': 1,
        'burst_last': 2},
    95: {
        'subswath': 'IW1',
        'burst_first': 1,
        'burst_last': 2},
    146: {
        'subswath': 'IW3',
        'burst_first': 2,
        'burst_last': 3},
    }

In [ ]:
subswaths = {
    22: 'IW2',
    73: 'IW1',
    95: 'IW1',
    146:'IW3'
}
#### Suitable bursts for individual products
bursts = {
    22: [
        [6,7], [1,2], [6,7], [1,2], [6,7], [1,2], [6,7], [1,2], [1,2], [1,2],
        [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2],
        [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2]
        ],
    73: [[1,2]] * 30,
    95: [
        [1,2], [1,2], [5,6], [1,2], [5,6], [1,2], [5,6], [1,2], [1,2], [1,2],
        [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2],
        [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2], [1,2]
        ],
    146: [[2,3]] * 30
}

In [ ]:
# create list of data directories to process and corresponding list of parameters to the TOPSAR_split function
years = (2021,)
relative_orbs = (22, 73, 95, 146)

# automate list creation based on years and ROs
data_dir_list, out_dir_list, subswaths_list, bursts_list = [], [], [], []
for year in years:
    for relative_orb in relative_orbs:
        data_dir_list.append(f'{ROOT_DIR}/s1_download/{year}/{relative_orb}')
        out_dir_list.append(f'{ROOT_DIR}/s1_coherence/{year}/{relative_orb}')
        subswaths_list.append(subswaths[relative_orb])
        bursts_list.append(bursts[relative_orb])

In [ ]:
time_a = perf_counter()

for data_dir, out_dir, subswath, bursts in zip(data_dir_list, out_dir_list, subswaths_list, bursts_list):
    
    files_list = glob('S1*.zip', root_dir=data_dir)
    files_list.sort(key=lambda x : x[17:25])
    dates_list = (filename[17:25] for filename in files_list)
    print(f'Working on datasets in folder {data_dir}')

    # preprocess master slc
    master_in = join(data_dir, files_list[0])
    master_gpt = join(xmls_dir, f'master_{basename(data_dir)}.xml')
    master_preprocessed = join(temp_dir, f'master_{basename(data_dir)}')
    create_master_gpt(master_gpt, master_in, master_preprocessed, subswath, bursts[0])
    gpt(xmlfile=master_gpt, tmpdir=temp_dir)
    print('Preprocessed common master image for coregistration.')
    
    for idx in range(len(files_list)-1):
        time_aa = perf_counter()
        date_1, date_2 = files_list[idx][17:25], files_list[idx+1][17:25]
        print(f'Computing coherence between images from {date_1} and {date_2}')
        
        # check if the time differnece between computtaions is not too small
        diff = int(str(datetime.strptime(date_2, "%Y%m%d") - datetime.strptime(date_1, "%Y%m%d")).split()[0])
        if diff < 6:
            warnings.warn(f'Computing coherence between observations only {diff} days apart. Is this correct?')
    
        graph_path = join(xmls_dir, f'coherence_{date_1}_{date_2}.xml')
        in_filepath_one = join(data_dir, files_list[idx])
        in_filepath_two = join(data_dir, files_list[idx+1])
        out_path = join(out_dir, f'coh_{date_1}_{date_2}')
    
        create_coherence_gpt(graph_path, in_master, in_filepath_one, in_filepath_two, dtm_path, out_path, subswath, [bursts[idx], bursts[idx+1]])
        gpt(xmlfile=graph_path, tmpdir=temp_dir, groups=groupbyWorkers(graph_path, n=2))

        print(f'Done in {(perf_counter() - time_aa)/60:.2f} minutes')

rmtree(temp_dir)
print(f'Finished computing all coherences in {(perf_counter()-time_a)/3600:.2f} hours.')

In [ ]:
# experimenting with time diff to warn about coherences that are not far enough apart
a, b = '20210403', '20210409'
diff = int(str(datetime.strptime(b, "%Y%m%d") - datetime.strptime(a, "%Y%m%d")).split()[0])
if diff > 6:
    print('hi')